In [ ]:
import tiktoken
import pandas as pd
from functools import partial

def num_tokens_from_string(string: str, model_name: str) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
cost_per_token = .15/1e6
model_name = 'gpt-4o-mini'
queries_per_sentence = 5

prompt1 = "The following sentences as from an academic paper, and cites a foundation model indicated by these <cite> citation brackets </cite>. These sentences have been identified as explicitly using the model, either by fine-tuning or using it out of the box, but we'd like to identify whether the authors were the ones doing so. Pretend you are the authors of the paper from which these sentences are pulled. Based on the following sentences, respond in the JSON format (without using ```json```) as {{\"response\": bool}} whether you were the one who used the model (true) or whether someone else did (false). Give one response for the whole set of sentences, without explanation. The sentence is as follows: {sentence}"

In [ ]:
path = '/Users/Alex/Desktop/2.FutureTech/OSFM/data/premicrosoft_meeting_classified.csv'
df = pd.read_csv(path)

extends_uses_mask = df['classification'].apply(lambda s: s in {'uses', 'extends'})
ignore_mask = (df['multisentence'].apply(lambda s: s.find('\\begin{tab') != -1)) | (df['strippedModelKey'] == 'ase')


In [ ]:
count_total_tokens = lambda s: num_tokens_from_string(prompt1.format(sentence = s), model_name=model_name) 
total_tokens = df[extends_uses_mask & ~ignore_mask]['multisentence'].apply(count_total_tokens).sum()
total_cost = cost_per_token * queries_per_sentence * total_tokens
print(f"${round(total_cost, 2)}")